# Jupyter and three.js
In this notebook we will attempt to illustrate the possibility of taking raw climate data and rendering it as part of an interactive 3D visualisation.

## Data Loading
Firstly we load some Met Office data using iris...  
We are using a single cube for this demo

In [ ]:
import iris
data = iris.load("./data/precip.nc")
print(data[0])

## Data Processing
Here we turn our data into a 1 dimensional RGBA array.  
To keep things simple, we are only packing data into the red (R) channel for now.  
This method of texture building expects a positive integer value in each channel between 0 and 255.  
As our data are probabilities between 0 and 1 we can simply map it into the desired range by multiplying each value by 255, then making it an int to remove the decimals.

In [ ]:
cdata = list(data[0].data.flatten())
rgba = []
for c in cdata:
    r = int(c*255)
    b, g, a = 0, 0, 255
    rgba += [r, g, b, a]

## Data Transfer
Here we attach our python array object to the js `window` object and call the new property `data`. This will allow us to use our data inside our javascript functions later on.

In [ ]:
import json
from IPython.display import Javascript

javascript = 'window.data={};'.format(json.dumps(rgba))
Javascript(javascript)


## JS Dependencies
Here we are loading in our js dependendencies.   
Once this is done there will be a global `THREE` object in the scope we can use to build our visualisation.  

Not sure why we have to use 2 differing methods but it was the only way we could currently make it work. 

In [ ]:
%%javascript
requirejs.config({
    paths: {
        T: ['//cdnjs.cloudflare.com/ajax/libs/three.js/r83/three']
    }
});
require(['T'], function(T) {
    window.THREE = T;
});

In [ ]:
%%html
<script src="https://s3-eu-west-1.amazonaws.com/molab-threejs/OrbitControls.js" type="text/javascript"></script>

## Shaders
These are our shaders.
They will be used to determine the final appearance of our dataset on the globe.

In [1]:
%%html

<script type="x-shader/x-vertex" id="my_vert">
uniform vec2 offset;
varying vec2 vUv;

void main(){

  vUv = uv;

  // apply longitude offset
  vUv.x = vUv.x + offset.x;
  if (vUv.x > 1.0) {
    vUv.x = vUv.x - 1.0;
  }

  gl_Position = projectionMatrix * modelViewMatrix * vec4( position, 1.0 );
}
</script>

<script type="x-shader/x-fragment" id="my_frag">
uniform sampler2D dataTexture;
uniform vec2 dataShape;
uniform sampler2D colorMap;
varying vec2 vUv;

float remapValueToRange(float val, float low1, float high1, float low2, float high2) {
  if (val < 0.1) {
    return 0.0;
  }
  return low2 + (val - low1) * (high2 - low2) / (high1 - low1);
}

void main() {

  float dataValue = texture2D(dataTexture, vUv).r;
  vec4 colorLookup = texture2D(colorMap, vec2(dataValue, 0.5));
  vec4 color = vec4(colorLookup.r, colorLookup.g, colorLookup.b, remapValueToRange(dataValue, 0.0, 1.0, 0.3, 0.8));
  gl_FragColor = color;

}
</script>

## HTML
Here we create our HTML parent element that we will render our graphics inside of.

In [ ]:
%%javascript
element.append("<div id='content'></div>");

## JS
Our ThreeJS javascript, where the scene and meshes are built.

In [ ]:
%%javascript

(function() {
    
    var parentDiv = document.getElementById("content");
    var controls, camera, scene, ambientLight, directionalLight, renderer;
    var textureLoader;
    var width = 800;
    var height = 400;
    
    cleanUp();
    init();
    animate();
    
    function init(){
        textureLoader = new THREE.TextureLoader();
        
        scene = new THREE.Scene();
        
        camera = new THREE.PerspectiveCamera( 75, width / height, 0.01, 1000 );
        camera.position.z = 2;
        
        renderer = new THREE.WebGLRenderer({antialias: true});
        renderer.setSize(width, height);
        renderer.setClearColor('black');
        
        ambientLight = new THREE.AmbientLight(0xffffff, 0.2); // soft white light
        scene.add(ambientLight);
        
        directionalLight = new THREE.DirectionalLight(0xbbbbbb, 0.8);
        directionalLight.position.copy(camera.position);
        scene.add(directionalLight);
        
        // add some controls
        controls = new THREE.OrbitControls(camera, renderer.domElement);
        controls.enablePan = false;
        controls.minDistance = 1.2;
        controls.maxDistance = 3;
        controls.enableDamping = true;
        controls.dampingFactor = 0.15;
        controls.rotateSpeed = 0.3;
        controls.autoRotate = true;
        controls.autoRotateSpeed = 0.2;
        // sync camera and directional light so we can see what we're doing!!
        controls.addEventListener('change', () => directionalLight.position.copy(camera.position));
        
        //put stuff on the screen
        parentDiv.appendChild( renderer.domElement ) ;
        
        
        addGlobeMesh();
        addDataMesh();
    }  
    
    
    function animate(time){
        window.requestAnimationFrame( animate ); 
        renderer.render( scene, camera);
    }
    
    
    function addGlobeMesh() {
        var sat = textureLoader.load('./textures/sat.jpg', texture => texture);
        var bump = textureLoader.load('./textures/bump.jpg', texture => texture);
        var specular = textureLoader.load('./textures/specular.png', texture => texture);

        var geometry = new THREE.SphereGeometry(1, 32, 32);
        var material = new THREE.MeshPhongMaterial({
          map: sat,
          bumpMap: bump,
          bumpScale: 0.015,
          specularMap: specular,
          specular: new THREE.Color('grey'),
          shininess: 10
        });
        var mesh = new THREE.Mesh(geometry, material);

        scene.add(mesh);
    }
    
    
    function getPyDataTexture() {
        var raw = Uint8Array.from(window.data);
        var texture = new THREE.DataTexture(raw,800,600,THREE.RGBAFormat);
        texture.magFilter = THREE.LinearFilter;
        texture.minFilter = THREE.LinearFilter;
        texture.needsUpdate = true;
        return texture;
    }
    
    function addDataMesh() {
        var colorMap = textureLoader.load('./textures/viridis.png', texture => texture);
        
        var geometry = new THREE.SphereGeometry(1.01, 32, 32);
        var material = new THREE.ShaderMaterial({
          uniforms: {
            offset: {
              type: 'v2',
              value: new THREE.Vector2(0.5, 0.0)
            },
            dataTexture: {
              type: 't',
              value: getPyDataTexture()
            },
            dataShape: {
              type: 'v2',
              value: new THREE.Vector2(800, 600)
            },
            colorMap: {
              type: 't',
              value: colorMap
            }
          },
          vertexShader: $('#my_vert').text(),
          fragmentShader: $('#my_frag').text(),
          transparent: true   // required if you want to alter alpha levels in shader
        });
        var mesh = new THREE.Mesh(geometry, material);
        scene.add(mesh);
    }
    
    
    function cleanUp() {
        if(parentDiv){
            while (parentDiv.firstChild) {
                parentDiv.removeChild(parentDiv.firstChild);
            }
        }
    }
    
})();